In [1]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import callbacks 
import matplotlib.pyplot as plt

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import os
import csv
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
root_dir='/'
train_dir = '/home/jupyter/theSimpsons-train/train'
test_dir = '/home/jupyter/theSimpsons-test/test'

In [4]:
# %matplotlib inline
# rows = 17
# cols = 6
# fig, ax = plt.subplots(rows, cols, frameon=False, figsize=(15, 25))
# food_dirs = os.listdir(train_dir)
# for i in range(rows):
#     for j in range(cols):
#         food_dir = food_dirs[(i*cols + j)%101]
#         all_files = os.listdir(os.path.join(train_dir, food_dir))
#         img = plt.imread(os.path.join(train_dir, food_dir, all_files[3]))
#         ax[i][j].imshow(img)
#         ax[i][j].text(0, -20, food_dir, size=10, rotation=0, ha="left", va="top", 
#                 bbox=dict(boxstyle="round", ec=(0, .6, .1), fc=(0, .7, .2)))
# plt.setp(ax, xticks=[], yticks=[])
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [5]:
batch_size = 16
data_seed = 120
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # width_shift_range=0.3,
    # height_shift_range=0.3,
    # shear_range=0.2,
    # zoom_range=0.2,
    rotation_range=15,
    validation_split=0.2,
    channel_shift_range=10,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(221, 221), 
    color_mode = "rgb",
    batch_size= batch_size,
    class_mode='categorical',
    subset='training',
    seed=data_seed,
    shuffle = True)

Found 77575 images belonging to 50 classes.


In [6]:
valid_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(221, 221), 
    color_mode = "rgb",
    batch_size= batch_size,
    class_mode='categorical',
    subset='validation',
    seed=data_seed,
    shuffle = True)

Found 19372 images belonging to 50 classes.


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(221, 221), 
    color_mode = "rgb",
)

Found 96947 images belonging to 50 classes.


In [8]:
# base_model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
# base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
# base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(221, 221, 3))
#base_model = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
#base_model = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(150, 150, 3))

In [10]:
init_weight = keras.initializers.GlorotUniform(seed=49)
model = models.Sequential()

# CNN
# model.add(layers.Conv2D(128, 3, input_shape=(221, 221, 3)))
# model.add(layers.MaxPooling2D(3,3))
# model.add(layers.Conv2D(32, 3))
# model.add(layers.MaxPooling2D(3,3))
# model.add(layers.Conv2D(32, 3))
# # model.add(layers.MaxPooling2D(3,3))
# model.add(layers.Flatten())

# # DNN
# model.add(layers.Dense(256, activation='relu'))
# # model.add(layers.Dropout(0.5))
# model.add(layers.Dense(128, activation='relu'))
# # model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(50, activation='softmax'))
model.add(Conv2D(64,(3,3),padding='same', input_shape=(221, 221, 3), activation='relu'))
model.add(BatchNormalization())
#Conv 2
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Conv 3
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
#Conv 4
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Conv 5
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
#Conv 6
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(50, activation='softmax'))


# callbacks
my_callbacks = [
    callbacks.EarlyStopping(patience=4, monitor='val_accuracy'),
    callbacks.ModelCheckpoint(
        'model.h5',
        monitor='val_accuracy',
        verbose=3,
        save_best_only=True,
        save_weight_only=True,
        mode='auto',
        period=1
    )
]

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 221, 221, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 221, 221, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 221, 221, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 221, 221, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 128)    

In [11]:
opt = Adam(learning_rate=0.001)
# opt = SGD(lr=0.01)
model.compile(optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_steps = valid_generator.samples // batch_size,
    validation_data=valid_generator,
    callbacks=my_callbacks,
    use_multiprocessing=True,
    workers=6,
    epochs=100
)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
4848/4848 [==============================] - 828s 170ms/step - loss: 2.4392 - accuracy: 0.3868 - val_loss: 1.4872 - val_accuracy: 0.6052

Epoch 00001: val_accuracy improved from -inf to 0.60522, saving model to model.h5
Epoch 2/100
4848/4848 [==============================] - 823s 170ms/step - loss: 1.1126 - accuracy: 0.7004 - val_loss: 0.7034 - val_accuracy: 0.8129

Epoch 00002: val_accuracy improved from 0.60522 to 0.81291, saving model to model.h5
Epoch 3/100
4848/4848 [==============================] - 823s 170ms/step - loss: 0.7418 - accuracy: 0.7960 - val_loss: 1.0625 - val_accuracy: 0.7368

Epoch 00003: val_accuracy did not improve from 0.81291
Epoch 4/100
4848/4848 [==============================] - 823s 170ms/step - loss: 0.5732 - accuracy: 0.8402 - val_loss: 0.3966 - val_accuracy: 0.8976

Epoch 00004: val_accuracy improved from 0.81291 to 0.89762, saving model to model.h5
Epoch 5/100
4848/4848 [==============================] - 823s 170ms/step - loss: 0.4603 - acc

In [13]:
# Create {ID, filename} dictionary {'1269': '1269.jpg', '3863': '3863.jpg',.....}
test_dict = {}
for root, dirs, files in os.walk(test_dir):
    for filename in files:
        test_id, file_ext = os.path.splitext(filename)
        test_dict[test_id] = filename

In [14]:
simpsons_dict = {}
dirs = os.listdir(train_dir)
dirs = sorted(dirs)

for dir in dirs:
    simpsons_dict[dirs.index(dir)] = dir
simpsons_dict

{0: 'abraham_grampa_simpson',
 1: 'agnes_skinner',
 2: 'apu_nahasapeemapetilon',
 3: 'barney_gumble',
 4: 'bart_simpson',
 5: 'brandine_spuckler',
 6: 'carl_carlson',
 7: 'charles_montgomery_burns',
 8: 'chief_wiggum',
 9: 'cletus_spuckler',
 10: 'comic_book_guy',
 11: 'disco_stu',
 12: 'dolph_starbeam',
 13: 'duff_man',
 14: 'edna_krabappel',
 15: 'fat_tony',
 16: 'gary_chalmers',
 17: 'gil',
 18: 'groundskeeper_willie',
 19: 'homer_simpson',
 20: 'jimbo_jones',
 21: 'kearney_zzyzwicz',
 22: 'kent_brockman',
 23: 'krusty_the_clown',
 24: 'lenny_leonard',
 25: 'lionel_hutz',
 26: 'lisa_simpson',
 27: 'lunchlady_doris',
 28: 'maggie_simpson',
 29: 'marge_simpson',
 30: 'martin_prince',
 31: 'mayor_quimby',
 32: 'milhouse_van_houten',
 33: 'miss_hoover',
 34: 'moe_szyslak',
 35: 'ned_flanders',
 36: 'nelson_muntz',
 37: 'otto_mann',
 38: 'patty_bouvier',
 39: 'principal_skinner',
 40: 'professor_john_frink',
 41: 'rainier_wolfcastle',
 42: 'ralph_wiggum',
 43: 'selma_bouvier',
 44: 'side

In [15]:
from PIL import Image
import numpy as np
from skimage import transform
def load_img(filename, target_w=150, target_h=150):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (target_w, target_h, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image


In [16]:
# Read images in order and make predictions
model = keras.models.load_model('model.h5')
results = []

for i in range(1, len(test_dict) + 1):
    img = load_img(test_dir + "/" + test_dict[str(i)], 221, 221)
    ret = model.predict(img)
    print("{:5d}: {:30s}".format(i, simpsons_dict[np.argmax(ret)]), end="\r")
    results.append(np.argmax(ret))

In [17]:
# Print results in CSV format and upload to Kaggle
with open('pred_results.csv', 'w') as f:
    f.write('id,character\n')
    for i in range(len(results)):
        f.write(str(i+1) + ',' + str(simpsons_dict[results[i]]) + '\n')

In [18]:
# Download your results!
from IPython.display import FileLink
FileLink('pred_results.csv')

/home/jupyter/pred_results.csv